In [421]:
# Import required libraries.
import  tensorflow as tf
import  keras

from    keras.models import Sequential
from    keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization
from    keras.utils import to_categorical
from    keras.callbacks import EarlyStopping

# Helper libraries.
import  numpy as np
import  sklearn
from    sklearn.model_selection import train_test_split

# Libraries for plotting.
import  matplotlib
import  matplotlib.pyplot as plt
from    PIL import Image
from    IPython.display import display

# Other
import gc

In [422]:
# Import required libraries.
import  tensorflow as tf
import  keras

from    keras.models import Sequential
from    keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization
from    keras.utils import to_categorical
from    keras.callbacks import EarlyStopping

# Helper libraries.
import  numpy as np
import  sklearn
from    sklearn.model_selection import train_test_split

# Libraries for plotting.
import  matplotlib
import  matplotlib.pyplot as plt
from    PIL import Image
from    IPython.display import display

# Other
import gc

In [423]:
print(tf.__version__)

In [424]:
# Test for GPU and determine what GPU we have.
# Modified by student to remove warningen caused on local machine.

import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if not tf.config.list_physical_devices('GPU'):
	print("No GPU was detected. CNNs can be very slow without a GPU.")
	if IN_COLAB:
		print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
else:
	os.system('nvidia-smi -L')

In [425]:
gpu_devices = tf.config.list_physical_devices('GPU')

if gpu_devices:
    details = tf.config.experimental.get_device_details(gpu_devices[0])
    compute_capability=details.get('compute_capability')
    print("Compute capability:",compute_capability)
    if compute_capability[0]>6:
        print("Turn on mixed_float16")
        policy = tf.keras.mixed_precision.Policy('mixed_float16')
        tf.keras.mixed_precision.set_global_policy(policy)
else:
    print("No GPU detected.")

In [426]:
# Modify GPU memory allocator to try to prevent full GPU memory.
# This can in some cases be counter productive!
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [427]:
# Install Keras Tuner via PIP (if in colab).
if IN_COLAB:
    os.system('pip install keras_tuner')

import keras_tuner
print('Keras Tuner version:', keras_tuner.__version__)

In [428]:
import zipfile

# Function to load a ZIP file from Google Drive.
def load_zip_file(zip_path):
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Extract to a specific directory
            zip_ref.extractall('/content/Model/dataset')
            print(f"ZIP file '{zip_path}' loaded successfully.")
    except FileNotFoundError:
        print(f"Error: ZIP file not found at '{zip_path}'.")
    except zipfile.BadZipFile:
        print(f"Error: Invalid ZIP file at '{zip_path}'.")

# If in Colab, load the ZIP file from Google Drive.
if IN_COLAB:
    print("Loading and extracting dataset ZIP-file form Drive.")
    load_zip_file("drive/MyDrive/Studies/KTH/Courses/II143X/Model Creation/Data/output_frames.zip")
else:
    print("No Colab detected, using pre-existing files.")

extracted_path = "output_frames"

In [429]:
dataset_array = []
dataset_classes = []
num_of_classes = 0
for directory in os.listdir(extracted_path):
    if os.path.isdir(os.path.join(extracted_path, directory)):
        num_of_classes += 1
        for file in os.listdir(os.path.join(extracted_path, directory)):
            if file.endswith(".txt"):
                dataset_array.append(os.path.join(extracted_path, directory, file))
                dataset_classes.append(directory)

In [430]:
print(len(dataset_array))
print(len(dataset_classes))

In [431]:
print(np.array(dataset_array).shape)
print(np.array(dataset_classes).shape)

In [432]:
print(dataset_array[0:2])

In [433]:
def processTXTFile(file_path):
    print(file_path)
    with open(file_path, 'r') as file:
        mfcc_txt = file.read()
        mfcc_data = np.array(eval(mfcc_txt)).T
    return mfcc_data

In [434]:
import librosa

def plot_mfcc(mfccs):
    print(f"MFCC shape: {mfccs.shape}")
    mfccs = mfccs
    librosa.display.specshow(mfccs, x_axis='time', sr=16000)
    plt.colorbar(format='%+2.0f dB')
    plt.title('MFCC')
    plt.xlabel('Time')
    plt.ylabel('MFCC Coefficient')
    plt.show()

In [435]:
dataset_array[0]

In [436]:
#for i in range(0, dataset_array[0].__len__()):
#    plot_mfcc(dataset_array[0][i])
plot_mfcc(processTXTFile(dataset_array[0]))

In [437]:
from sklearn.preprocessing import LabelEncoder

# Split up the data into training and testing data.
x_train, x_temp, y_train, y_temp = train_test_split(dataset_array, dataset_classes, test_size=0.3, train_size=0.7)

# Further split the temporary set into validation and test sets.
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, train_size=0.5)

# Encode string labels to integers
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

print(num_of_classes)

y_train_cat = to_categorical(y_train_encoded, num_classes=num_of_classes)
y_val_cat = to_categorical(y_val_encoded, num_classes=num_of_classes)
y_test_cat = to_categorical(y_test_encoded, num_classes=num_of_classes)

print(f"Train data shape: {np.array(x_train).shape}")
print(f"Train labels shape: {y_train_cat.shape}")
print(f"Validation data shape: {np.array(x_val).shape}")
print(f"Validation labels shape: {y_val_cat.shape}")
print(f"Test data shape: {np.array(x_test).shape}")
print(f"Test labels shape: {y_test_cat.shape}")


In [438]:
x_train_tf = tf.data.TextLineDataset(x_train)

def processTXTFile_tf(file_path):
    file_path = file_path.numpy().decode('utf-8')
    data = np.concatenate(processTXTFile(file_path), axis=0)
    return data

x_train_tf = x_train_tf.interleave(
    lambda file: tf.data.Dataset.from_tensors(tf.py_function(processTXTFile_tf, [file], [tf.float32])),
    cycle_length=1,  # Controls how many files are read in parallel (set to 1 for strict on-demand processing)
    num_parallel_calls=tf.data.AUTOTUNE
)

# Create a dataset for y_train
y_train_tf = tf.data.Dataset.from_tensor_slices(y_train_cat)

# Combine x_train_tf and y_train_tf into a single dataset
train_tf = tf.data.Dataset.zip((x_train_tf, y_train_tf))

# Batch and prefetch the dataset
train_tf = train_tf.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_dataset = train_tf.batch(1).prefetch(tf.data.AUTOTUNE)

In [439]:
x_val_tf = tf.data.TextLineDataset(dataset_array)

def processTXTFile_tf(file_path):
    file_path = file_path.numpy().decode('utf-8')
    data = np.concatenate(processTXTFile(file_path), axis=0)
    return data

x_val_tf = x_val_tf.interleave(
    lambda file: tf.data.Dataset.from_tensors(tf.py_function(processTXTFile_tf, [file], [tf.float32])),
    cycle_length=1,  # Controls how many files are read in parallel (set to 1 for strict on-demand processing)
    num_parallel_calls=tf.data.AUTOTUNE
)

# Create a dataset for y_train
y_val_tf = tf.data.Dataset.from_tensor_slices(y_val_cat)

# Combine x_train_tf and y_train_tf into a single dataset
val_tf = tf.data.Dataset.zip((x_val_tf, y_val_tf))

# Batch and prefetch the dataset
val_tf = val_tf.batch(32).prefetch(tf.data.AUTOTUNE)

In [440]:
print(y_train_cat.shape)

In [441]:
print(f"Classes: {format(dataset_classes)}")

In [442]:
# Creation of the sequential model and all layers

input_shape = (25, 16, 1)

sequential_model = Sequential([
    Conv2D(filters = 64, kernel_size = (3,3), activation = "relu", input_shape = input_shape, padding = 'same'),
    Conv2D(filters = 64, kernel_size = (2,2), activation = "relu", input_shape = input_shape, padding = 'same'),
    MaxPool2D(pool_size=(2,2)),
    Dropout(0.4),
    Conv2D(filters = 128, kernel_size = (3,3), activation = "relu", input_shape = input_shape, padding = 'same'),
    MaxPool2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation = "relu"),
    Dense(64, activation = "relu"),
    Dropout(0.6),
    Dense(units = num_of_classes, activation = "softmax")
])


In [443]:
sequential_model.summary()

In [444]:
sequential_model.compile(optimizer = "Adam", loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])

In [445]:
def printHistory(history):
  epochs = len(history.history['loss']) # Collect the number of epochs run based on the amount of loss value under history.

  epochrange = range(1, epochs + 1)
  train_acc = history.history['categorical_accuracy']
  val_acc = history.history['val_categorical_accuracy']

  train_loss = history.history['loss']
  val_loss = history.history['val_loss']

  plt.plot(epochrange, train_acc, 'bo', label='Training acc')
  plt.plot(epochrange, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy (modell 1)')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.show()

  plt.plot(epochrange, train_loss, 'bo', label='Training loss')
  plt.plot(epochrange, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss (modell 1)')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

In [446]:
def fitModel(model, train_tf, val_tf, epochs, batch_size, doPrintHistory):
    history = model.fit(train_tf, validation_data=val_tf, epochs=epochs, batch_size=batch_size)
    if doPrintHistory: printHistory(history)

In [ ]:
# Check size of what train_tf is
print(train_tf)

In [447]:
fitModel(sequential_model, train_tf, val_tf, 10, 32, True)